In [35]:
import os
import json
import random
import torch
import requests
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer, util

In [36]:
load_dotenv()
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

if TOGETHER_API_KEY is None:
    raise ValueError("❌ API key not loaded. Check your .env file name or path.")

In [37]:
class SemanticChatbot:
    def __init__(
        self,
        intents_path,
        model_name='all-MiniLM-L6-v2',
        embeddings_path="embeddings.pt",
        together_model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
    ):
        self.intents_path = intents_path
        self.embeddings_path = embeddings_path
        self.together_model_name = together_model_name
        self.model = SentenceTransformer(model_name)

        self.patterns = []
        self.responses = []
        self.embeddings = None

        self._load_data()
        self._load_or_build_embeddings()

    def _load_data(self):
        with open(self.intents_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        self.patterns = [item["pattern"] for item in data["intents"]]
        self.responses = [item["responses"] for item in data["intents"]]

    def _load_or_build_embeddings(self):
        if os.path.exists(self.embeddings_path):
            print(f"📥 Chargement des embeddings depuis {self.embeddings_path}...")
            self.embeddings = torch.load(self.embeddings_path)
        else:
            print("🔄 Encodage des patterns avec SBERT...")
            self.embeddings = self.model.encode(self.patterns, convert_to_tensor=True)
            torch.save(self.embeddings, self.embeddings_path)
            print(f"💾 Embeddings sauvegardés dans {self.embeddings_path}")

    def build_prompt(self, user_input, top_k_hits):
        examples = ""
        for i, hit in enumerate(top_k_hits[:3]):
            idx = hit["corpus_id"]
            question = self.patterns[idx]
            answer = random.choice(self.responses[idx])
            examples += f"User: {question}\nBot: {answer}\n\n"

        prompt = (
            "You are a helpful assistant. Below are a few example conversations.\n"
            "Use them to respond to the final user input in a concise and natural way.\n\n"
            f"{examples}"
            f"User: {user_input}\nBot:"
        )
        return prompt

    def generate_with_together(self, prompt, max_tokens=150):
        headers = {
            "Authorization": f"Bearer {TOGETHER_API_KEY}",
            "Content-Type": "application/json"
        }

        data = {
            "model": self.together_model_name,
            "prompt": prompt,
            "max_tokens": max_tokens,
            "temperature": 0.7,
            "top_p": 0.9,
            "stop": ["User:", "Bot:"]
        }

        response = requests.post("https://api.together.xyz/v1/completions", headers=headers, json=data)

        if response.status_code != 200:
            print("❌ Error from Together API:", response.text)
            return "Désolé, j'ai eu un problème pour répondre."

        return response.json()["choices"][0]["text"].strip()

    def get_response(self, user_input, top_k=5):
        query_embedding = self.model.encode(user_input, convert_to_tensor=True)
        hits = util.semantic_search(query_embedding, self.embeddings, top_k=top_k)[0]

        if not hits:
            return "Je ne suis pas sûr de comprendre."

        prompt = self.build_prompt(user_input, hits)
        return self.generate_with_together(prompt)


In [38]:
if __name__ == '__main__':
    print("🚀 Chargement du chatbot sémantique avec Together.ai...")
    bot = SemanticChatbot("dailydialog_sbert.json")

    print("\n🤖 Chatbot prêt ! Tape une phrase (ou /quit pour quitter)\n")
    while True:
        message = input("🗣️ You: ")
        if message.strip().lower() == "/quit":
            print("👋 Bye!")
            break

        response = bot.get_response(message)
        print(f"🤖 Bot: {response}")


🚀 Chargement du chatbot sémantique avec Together.ai...
📥 Chargement des embeddings depuis embeddings.pt...

🤖 Chatbot prêt ! Tape une phrase (ou /quit pour quitter)

🤖 Bot: Ireland is a country located in Europe. It is not part of Britain or America.
🤖 Bot: Neither actually . New Zealand is an independent nation with its own government and monarch .
👋 Bye!
